In [1]:
%matplotlib inline

import os
import sys
import glob
import pprint

import numpy as np
import scipy as sp
import pandas as pd
import scipy.stats as sps
import statsmodels.api as sm

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.mlab as mlab
import matplotlib.ticker as ticker
import matplotlib.gridspec as gridspec

import radical.utils as ru
import radical.pilot as rp
import radical.analytics as ra

from IPython.display import display

pd.set_option('expand_frame_repr', False)

/Users/mturilli/Virtualenvs/RADICAL-ANALYTICS/lib/python2.7/site-packages/radical/utils/atfork/stdlib_fixer.py:58: UserWarning: logging module already imported before fixup.
  warnings.warn('logging module already imported before fixup.')


In [2]:
# Global configurations
# ---------------------

# Use LaTeX and its body font for the diagrams' text.
mpl.rcParams['text.usetex'] = True 
mpl.rcParams['font.family'] = 'serif'
mpl.rcParams['font.serif']  = ['Nimbus Roman Becker No9L']

# Use thinner lines for axes to avoid distractions.
mpl.rcParams['axes.linewidth']    = 0.75
mpl.rcParams['xtick.major.width'] = 0.75
mpl.rcParams['xtick.minor.width'] = 0.75
mpl.rcParams['ytick.major.width'] = 0.75
mpl.rcParams['ytick.minor.width'] = 0.75

# Do not use a box for the legend to avoid distractions.
mpl.rcParams['legend.frameon'] = False

# Helpers
# -------

# Use coordinated colors. These are the "Tableau 20" colors as 
# RGB. Each pair is strong/light. For a theory of color    
tableau20 = [(31 , 119, 180), (174, 199, 232), # blue   [ 0,1 ]
             (255, 127, 14 ), (255, 187, 120), # orange [ 2,3 ]
             (44 , 160, 44 ), (152, 223, 138), # green  [ 4,5 ]
             (214, 39 , 40 ), (255, 152, 150), # red    [ 6,7 ]
             (148, 103, 189), (197, 176, 213), # purple [ 8,9 ]
             (140, 86 , 75 ), (196, 156, 148), # brown  [10,11]
             (227, 119, 194), (247, 182, 210), # pink   [12,13]
             (127, 127, 127), (199, 199, 199), # gray   [14,15]
             (188, 189, 34 ), (219, 219, 141), # yellow [16,17]
             (23 , 190, 207), (158, 218, 229)] # cyan   [18,19]
  
# Scale the RGB values to the [0, 1] range, which is the format 
# matplotlib accepts.    
for i in range(len(tableau20)):  
    r, g, b = tableau20[i]  
    tableau20[i] = (r / 255., g / 255., b / 255.)    

# Return a single plot without right and top axes
def fig_setup():
    fig = plt.figure(figsize=(13,7))
    ax = fig.add_subplot(111)  
    ax.spines["top"].set_visible(False)  
    ax.spines["right"].set_visible(False)  
    ax.get_xaxis().tick_bottom()  
    ax.get_yaxis().tick_left()
    
    return fig, ax

In [29]:
def load_data(rdir):
    sessions = {}
    experiments = {}
    start = rdir.rfind(os.sep)+1
    for path, dirs, files in os.walk(rdir):
        folders = path[start:].split(os.sep)
        if len(path[start:].split(os.sep)) == 2:
            sid = os.path.basename(glob.glob('%s/*.json' % path)[0])[:-5]
            if sid not in sessions.keys():
                sessions[sid] = {}
            sessions[sid] = ra.Session(sid, 'radical.pilot', src=path)
            experiments[sid] = folders[0]
    return sessions, experiments

# Load experiments' dataset into ra.session objects
# stored in a DataFrame.
rdir = 'data/'
sessions, experiments = load_data(rdir)
sessions = pd.DataFrame({'session': sessions,
                         'experiment': experiments})

In [30]:
pd.set_option('display.max_rows', None)
len(sessions)
# pd.reset_option('display.max_rows')

72

In [5]:
for sid in sessions.index:
    sessions.ix[sid, 'TTC'] = sessions.ix[sid, 'session'].ttc

In [6]:
for sid in sessions.index:
    sessions.ix[sid, 'nunit'] = len(sessions.ix[sid, 'session'].filter(etype='unit', inplace=False).get())

In [7]:
# Model of TOTAL pilot durations.
ttpdm = {'TT_PILOT_PMGR_SCHEDULING': ['NEW'                   ,  'PMGR_LAUNCHING_PENDING'],
         'TT_PILOT_PMGR_QUEUING'   : ['PMGR_LAUNCHING_PENDING',  'PMGR_LAUNCHING'],
         'TT_PILOT_LRMS_SUBMITTING': ['PMGR_LAUNCHING'        ,  'PMGR_ACTIVE_PENDING'],
         'TT_PILOT_LRMS_QUEUING'   : ['PMGR_ACTIVE_PENDING'   ,  'PMGR_ACTIVE'],
         'TT_PILOT_LRMS_RUNNING'   : ['PMGR_ACTIVE'           , ['DONE',
                                                                 'CANCELED',
                                                                 'FAILED']]}

# Add total pilot durations to sessions' DF.
for sid in sessions.index:
    s = sessions.ix[sid, 'session'].filter(etype='pilot', inplace=False)
    for d in ttpdm.keys():
        sessions.ix[sid, d] = s.duration(ttpdm[d])

In [8]:
sessions

,experiment,session,TTC,nunit,TT_PILOT_LRMS_QUEUING,TT_PILOT_LRMS_SUBMITTING,TT_PILOT_PMGR_QUEUING,TT_PILOT_PMGR_SCHEDULING,TT_PILOT_LRMS_RUNNING
rp.session.radical.mingtha.017033.0007,exp1,<radical.analytics.session.Session object at 0...,1596.3935,16.0,448.9312,40.8441,0.0021,0.1115,1331.9196
rp.session.radical.mingtha.017033.0008,exp1,<radical.analytics.session.Session object at 0...,4904.5828,64.0,172.4000,32.6656,0.0019,0.1159,4716.4960
rp.session.radical.mingtha.017033.0009,exp1,<radical.analytics.session.Session object at 0...,2796.6535,32.0,235.8966,30.7743,0.0022,0.1041,2603.0602
rp.session.radical.mingtha.017034.0002,exp1,<radical.analytics.session.Session object at 0...,6110.7223,64.0,231.2612,25.5676,0.0021,0.1143,5939.8338
rp.session.radical.mingtha.017034.0003,exp1,<radical.analytics.session.Session object at 0...,1802.5047,16.0,229.3836,27.1441,0.0022,0.1173,1613.6261
rp.session.radical.mingtha.017034.0004,exp1,<radical.analytics.session.Session object at 0...,3428.5537,32.0,230.0107,26.4455,0.0020,0.1200,3223.5324
rp.session.radical.mingtha.017034.0005,exp1,<radical.analytics.session.Session object at 0...,2377.8985,16.0,247.5876,26.2371,0.0021,0.1347,2137.6839
rp.session.radical.mingtha.017034.0006,exp1,<radical.analytics.session.Session object at 0...,3419.9937,32.0,257.3524,25.5355,0.0021,0.1218,3203.5055
rp.session.radical.mingtha.017035.0003,exp1,<radical.analytics.session.Session object at 0...,1874.7339,16.0,269.2108,25.5982,0.0022,0.1030,1637.4107
rp.session.radical.mingtha.017035.0005,exp1,<radical.analytics.session.Session object at 0...,5680.6695,64.0,245.1551,25.7160,0.0020,0.1026,5473.1517


In [14]:
# Model of pilot durations.
pdm = {'PMGR_SCHEDULING': ['NEW'                   ,  'PMGR_LAUNCHING_PENDING'],
       'PMGR_QUEUING'   : ['PMGR_LAUNCHING_PENDING',  'PMGR_LAUNCHING'],
       'LRMS_SUBMITTING': ['PMGR_LAUNCHING'        ,  'PMGR_ACTIVE_PENDING'],
       'LRMS_QUEUING'   : ['PMGR_ACTIVE_PENDING'   ,  'PMGR_ACTIVE'],
       'LRMS_RUNNING'   : ['PMGR_ACTIVE'           , ['DONE',
                                                      'CANCELED',
                                                      'FAILED']]}

# DataFrame structure for pilot durations. 
pds = { 'pid': [],
        'sid': [],
        'experiment'     : [],
        'PMGR_SCHEDULING': [],
        'PMGR_QUEUING'   : [],
        'LRMS_SUBMITTING': [],
        'LRMS_QUEUING'   : [],
        'LRMS_RUNNING'   : []}

# Calculate the duration for each state of each 
# pilot of each run and Populate the DataFrame 
# structure.
for sid in sessions.index:
    s = sessions.ix[sid, 'session'].filter(etype='pilot', inplace=False)
    for p in s.list('uid'):
        sf = s.filter(uid=p, inplace=False)        
        pds['pid'].append(p)
        pds['sid'].append(sid)
        pds['experiment'].append(sessions.ix[sid, 'experiment'])
        for d in pdm.keys():
            if (not sf.timestamps(state=pdm[d][0]) or 
                not sf.timestamps(state=pdm[d][1])):
                pds[d].append(None)
                continue
            pds[d].append(sf.duration(pdm[d]))

# Populate the DataFrame.
pilots = pd.DataFrame(pds)

In [10]:
pilots

,LRMS_QUEUING,LRMS_RUNNING,LRMS_SUBMITTING,PMGR_QUEUING,PMGR_SCHEDULING,experiment,pid,sid
0,217.1893,1330.8070,40.8441,0.0021,0.1115,exp1,pilot.0002,rp.session.radical.mingtha.017033.0007
1,220.5954,1321.0085,40.8441,0.0021,0.1115,exp1,pilot.0003,rp.session.radical.mingtha.017033.0007
2,448.9312,1096.9671,40.8441,0.0021,0.1115,exp1,pilot.0000,rp.session.radical.mingtha.017033.0007
3,275.0083,1274.1006,40.8441,0.0021,0.1115,exp1,pilot.0001,rp.session.radical.mingtha.017033.0007
4,172.3086,4668.0595,32.6656,0.0019,0.1159,exp1,pilot.0002,rp.session.radical.mingtha.017033.0008
5,172.4000,4670.0795,32.6656,0.0019,0.1159,exp1,pilot.0003,rp.session.radical.mingtha.017033.0008
6,171.3112,4665.8320,32.6656,0.0019,0.1159,exp1,pilot.0000,rp.session.radical.mingtha.017033.0008
7,125.9835,4708.0529,32.6656,0.0019,0.1159,exp1,pilot.0001,rp.session.radical.mingtha.017033.0008
8,153.8039,2601.0478,30.7743,0.0022,0.1041,exp1,pilot.0002,rp.session.radical.mingtha.017033.0009
9,235.8966,2517.9505,30.7743,0.0022,0.1041,exp1,pilot.0003,rp.session.radical.mingtha.017033.0009


In [15]:
def parse_osg_hostid(hostid):
    '''
    Heuristic: eliminate node-specific information from hostID.
    '''
    domain = None
    
    # Split domain name from IP.
    host = hostid.split(':')

    # Split domain name into words.
    words = host[0].split('.')

    # Get the words in the domain name that do not contain 
    # numbers. Most hostnames have no number but there are 
    # exceptions.
    literals = [l for l in words if not any((number in set('0123456789')) for number in l)]            

    # Check for exceptions: 
    # a. every word of the domain name has a number
    if len(literals) == 0:

        # Some hostname use '-' instead of '.' as word separator.
        # The parser would have returned a single word and the 
        # any of that word may have a number.
        if '-' in host[0]:
            words = host[0].split('-')
            literals = [l for l in words if not any((number in set('0123456789')) for number in l)]

            # FIXME: We do not check the size of literals. 
            domain = '.'.join(literals)

        # Some hostnames may have only the name of the node. We
        # have to keep the IP to decide later on whether two nodes
        # are likely to belong to the same cluster.
        elif 'n' in host[0] or 'nod' in host[0]:
            domain = '.'.join(host)

        # The hostname is identified by an alphanumeric string
        else:
            domain = '.'.join(host)

    # Some hostnames DO have numbers in their name.
    elif len(literals) == 1:
        domain = '.'.join(words[1:])

    # Some hostname are just simple to parse.
    else:
        domain = '.'.join(literals)

    return domain

In [19]:
for pix in pilots.index:
    sid = pilots.ix[pix,'sid']
    pid = pilots.ix[pix,'pid']
    pls = sessions.ix[sid, 'session'].filter(uid=pid, inplace=False).get(etype=['pilot'])
    if len(pls) != 1:
        print "Error: session filter on uid returned multiple pilots"
        break
    if pls[0].cfg['hostid']:
        domain = parse_osg_hostid(pls[0].cfg['hostid'])
    else:
        domain = np.nan
    # print '%s: %s: %s: %s' % (pix, sid, pid, domain)
    pilots.ix[pix,'domain'] = domain

pilots

,LRMS_QUEUING,LRMS_RUNNING,LRMS_SUBMITTING,PMGR_QUEUING,PMGR_SCHEDULING,experiment,pid,sid,domain
0,217.1893,1330.8070,40.8441,0.0021,0.1115,exp1,pilot.0002,rp.session.radical.mingtha.017033.0007,ucsd.edu
1,220.5954,1321.0085,40.8441,0.0021,0.1115,exp1,pilot.0003,rp.session.radical.mingtha.017033.0007,ucsd.edu
2,448.9312,1096.9671,40.8441,0.0021,0.1115,exp1,pilot.0000,rp.session.radical.mingtha.017033.0007,b7c6cf8a3ae1.172.19.0.3
3,275.0083,1274.1006,40.8441,0.0021,0.1115,exp1,pilot.0001,rp.session.radical.mingtha.017033.0007,phys.uconn.edu
4,172.3086,4668.0595,32.6656,0.0019,0.1159,exp1,pilot.0002,rp.session.radical.mingtha.017033.0008,ucsd.edu
5,172.4000,4670.0795,32.6656,0.0019,0.1159,exp1,pilot.0003,rp.session.radical.mingtha.017033.0008,icecube.wisc.edu
6,171.3112,4665.8320,32.6656,0.0019,0.1159,exp1,pilot.0000,rp.session.radical.mingtha.017033.0008,b7c6cf8a3ae1.172.19.0.3
7,125.9835,4708.0529,32.6656,0.0019,0.1159,exp1,pilot.0001,rp.session.radical.mingtha.017033.0008,ucsd.edu
8,153.8039,2601.0478,30.7743,0.0022,0.1041,exp1,pilot.0002,rp.session.radical.mingtha.017033.0009,ucsd.edu
9,235.8966,2517.9505,30.7743,0.0022,0.1041,exp1,pilot.0003,rp.session.radical.mingtha.017033.0009,phys.uconn.edu


In [12]:
# Temporary: workaround for bug ticket \#15. Calculates 
# the number of active pilots by looking into the 
# length of the list returned by timestamp on the 
# PMGR_ACTIVE state.

for sid in sessions.index:
    sessions.ix[sid, 'npilot_active'] = len(sessions.ix[sid, 'session'].filter(etype='pilot', inplace=False).timestamps(state='PMGR_ACTIVE'))

In [26]:
# Add number of unique resorces per session.

for sid in sessions.index:
    sessions.ix[sid, 'n_unique_host'] = len(pilots[pilots['sid'] == sid]['domain'].unique())

In [28]:
len(sessions)

72